<a href="https://colab.research.google.com/github/dmswl0707/Dacon_AI_Competition/blob/main/%EC%98%81%ED%99%94_%EB%A6%AC%EB%B7%B0_%EA%B0%90%EC%84%B1%EB%B6%84%EB%A5%98_%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [224]:
import os
import pandas as pd
from google.colab import drive
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [225]:
!pip install konlpy

In [240]:
train = '/content/drive/My Drive/Colab Notebooks/dataset/train.csv'
test = '/content/drive/My Drive/Colab Notebooks/dataset/test.csv'

df = pd.read_csv(train)
testset = pd.read_csv(test)

df.head(3)

,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1
2,3,괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~,1


In [241]:
df.shape

(5000, 3)

In [242]:
# 긍정 부정 간 레이블 균형이 비슷함

df=df.drop(columns='id')
df.value_counts('label')

label
0    2564
1    2436
dtype: int64

In [243]:
df.isnull().sum()

document    0
label       0
dtype: int64

### 데이터 전처리
정규 표현식 제거, 불용어 제거, 토큰화, 인코딩 작업. 



In [244]:
# 정규 표현식 제거
df['document']=df['document'].str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]", "")
df.head(3)

,document,label
0,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯,1
2,괜찮은 음악영화가 또 나왔군요 따뜻한 겨울이 될 것 같아요,1


In [245]:
from tqdm import tqdm
from konlpy.tag import Okt

In [246]:
# 불용어 제거
# 불용어를 직접 정의할수도 있고, set(stop_words.split(" "))을 사용하기도 함

stop_words = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']
x_train = []
x_test = []
okt = Okt()

def remove_stopwords(list_, df):
  for sent in tqdm(df):
    token = okt.morphs(sent, stem=True)
    rem_stopwords = [word for word in token if not word in stop_words]
    list_.append(rem_stopwords)

In [247]:
# train set 가공
remove_stopwords(x_train, df['document'])

100%|██████████| 5000/5000 [00:15<00:00, 330.06it/s]


In [248]:
# test set 가공
remove_stopwords(x_test, testset['document'])

100%|██████████| 5000/5000 [00:13<00:00, 364.55it/s]


In [250]:
# 토큰화 작업
# 인덱스 확인하기
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)

In [251]:
vocab_size =len(tokenizer.word_index)
print(tokenizer.word_index)

{'영화': 1, '보다': 2, '없다': 3, '을': 4, '너무': 5, '이다': 6, '좋다': 7, '정말': 8, '재밌다': 9, '있다': 10, '진짜': 11, '다': 12, '만': 13, '적': 14, '최고': 15, '되다': 16, '로': 17, '않다': 18, '점': 19, '평점': 20, '같다': 21, '연기': 22, '내': 23, '아깝다': 24, '나오다': 25, '만들다': 26, '왜': 27, '아니다': 28, '감동': 29, '것': 30, '재미': 31, '지루하다': 32, '이렇다': 33, '재미있다': 34, '에서': 35, '게': 36, '재미없다': 37, '보고': 38, '나': 39, '인': 40, '그': 41, '시간': 42, '스토리': 43, '중': 44, '못': 45, '말': 46, '드라마': 47, '배우': 48, 'ㅋㅋ': 49, '요': 50, '안': 51, '거': 52, '감독': 53, '때': 54, '그냥': 55, '아': 56, '내용': 57, '생각': 58, '더': 59, '뭐': 60, '자다': 61, '최악': 62, 'ㅡㅡ': 63, '까지': 64, '사람': 65, '고': 66, '네': 67, '완전': 68, '주다': 69, '모르다': 70, '싶다': 71, 'ㅠㅠ': 72, '볼': 73, '수': 74, '이렇게': 75, '다시': 76, '가다': 77, '사랑': 78, '지': 79, '액션': 80, '돈': 81, 'ㅋㅋㅋ': 82, '이야기': 83, '하나': 84, '작품': 85, '알다': 86, '쓰레기': 87, '끝': 88, '이건': 89, '별': 90, '그렇다': 91, '차다': 92, '오다': 93, '마지막': 94, '명작': 95, '정도': 96, '저': 97, '아름답다': 98, '하고': 99, '걸': 100, '나다': 101, '남다': 

In [252]:
# 정수 인코딩 작업
tokenizer=Tokenizer(vocab_size)
tokenizer.fit_on_texts(x_train)
x_train=tokenizer.texts_to_sequences(x_train)

In [255]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
# 패딩의 길이 임의로 설정
x_train=pad_sequences(x_train, maxlen=20)
x_test=pad_sequences(x_train, maxlen=20)

y_train=df['label']

In [258]:
print(vocab_size)

6648


In [267]:
from tensorflow.keras.layers import Embedding, Dense,Flatten, Bidirectional,LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()
model.add(Embedding(vocab_size, 1000))
model.add(Bidirectional(LSTM(500, activation='elu', recurrent_dropout=0.1)))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.summary()



Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, None, 1000)        6648000   
                                                                 
 bidirectional_3 (Bidirectio  (None, 1000)             6004000   
 nal)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 1000)              0         
                                                                 
 dense_3 (Dense)             (None, 1)                 1001      
                                                                 
Total params: 12,653,001
Trainable params: 12,653,001
Non-trainable params: 0
_________________________________________________________________


In [269]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train, epochs=30, callback=[], batch_size=16)

TypeError: ignored